#Assignemt 4 - Cookie Cats A/B Test - Bayesian Analysis

###https://github.com/dustywhite7/econ8310-assignment4

In [3]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import beta
import arviz as az

df = pd.read_csv('https://github.com/dustywhite7/Econ8310/raw/master/AssignmentData/cookie_cats.csv')
print(df.head())
print("\n")
print(df['version'].value_counts())
print("\n")
print(df[['retention_1', 'retention_7']].mean())
print("\n")

   userid  version  sum_gamerounds  retention_1  retention_7
0     116  gate_30               3        False        False
1     337  gate_30              38         True        False
2     377  gate_40             165         True        False
3     483  gate_40               1        False        False
4     488  gate_40             179         True         True


version
gate_40    45489
gate_30    44700
Name: count, dtype: int64


retention_1    0.445210
retention_7    0.186065
dtype: float64




In [4]:
# Let's divide between control which is gate_30) and treatment=gate_40
control = df[df['version'] == 'gate_30']
treatment = df[df['version'] == 'gate_40']

# Counts for day1 retention
success_control_1 = control['retention_1'].sum()
trials_control_1  = len(control)

success_treat_1 = treatment['retention_1'].sum()
trials_treat_1  = len(treatment)

# Counts for day7 retention
success_control_7 = control['retention_7'].sum()
trials_control_7  = len(control)

success_treat_7 = treatment['retention_7'].sum()
trials_treat_7  = len(treatment)

print(f"day1 retention Control g30: {success_control_1}/{trials_control_1} = {success_control_1/trials_control_1:.4%}")
print(f"day1 retention Treatme g40: {success_treat_1}/{trials_treat_1} = {success_treat_1/trials_treat_1:.4%}")

print(f"day7 retention Control g30): {success_control_7}/{trials_control_7} = {success_control_7/trials_control_7:.4%}")
print(f"day7 retention Treatme g40): {success_treat_7}/{trials_treat_7} = {success_treat_7/trials_treat_7:.4%}")

day1 retention Control g30: 20034/44700 = 44.8188%
day1 retention Treatme g40: 20119/45489 = 44.2283%
day7 retention Control g30): 8502/44700 = 19.0201%
day7 retention Treatme g40): 8279/45489 = 18.2000%


Bayesian Model

In [5]:
# Weakly informative prior: Beta(1,1) = Uniform, but we'll use Beta(2,2) for slight regularization
prior_alpha = 2
prior_beta  = 2

# Posterior parameters
post_control_1 = beta(success_control_1 + prior_alpha, trials_control_1 - success_control_1 + prior_beta)
post_treat_1   = beta(success_treat_1 + prior_alpha,   trials_treat_1 - success_treat_1 + prior_beta)

post_control_7 = beta(success_control_7 + prior_alpha, trials_control_7 - success_control_7 + prior_beta)
post_treat_7   = beta(success_treat_7 + prior_alpha,   trials_treat_7 - success_treat_7 + prior_beta)

# Generate posterior samples
np.random.seed(42)
samples = 100_000

control_1_samples = post_control_1.rvs(samples)
treat_1_samples   = post_treat_1.rvs(samples)
control_7_samples = post_control_7.rvs(samples)
treat_7_samples   = post_treat_7.rvs(samples)

# Relative effect (treatment vs control)
relative_effect_1 = treat_1_samples / control_1_samples - 1
relative_effect_7 = treat_7_samples / control_7_samples - 1

absolute_effect_1 = treat_1_samples - control_1_samples
absolute_effect_7 = treat_7_samples - control_7_samples

Results:

In [6]:
print("DAY 1 Retention _____________________")
print(f"Control (gate_30) posterior mean: {control_1_samples.mean():.4%}")
print(f"Treatment (gate_40) posterior mean: {treat_1_samples.mean():.4%}")
print(f"Absolute difference: {absolute_effect_1.mean():.4%} percentage points")
print(f"Relative change: {relative_effect_1.mean():.2%}")
print(f"Probability that gate_40 is BETTER than gate_30: {(treat_1_samples > control_1_samples).mean():.4%}")
print(f"95% Credible Interval for relative effect: [{np.percentile(relative_effect_1, 2.5):.2%}, {np.percentile(relative_effect_1, 97.5):.2%}]")

print("\n")
print("DAY 7 Retention _____________________")
print(f"Control (gate_30) posterior mean: {control_7_samples.mean():.4%}")
print(f"Treatment (gate_40) posterior mean: {treat_7_samples.mean():.4%}")
print(f"Absolute difference: {absolute_effect_7.mean():.4%} percentage points")
print(f"Relative change: {relative_effect_7.mean():.2%}")
print(f"Probability that gate_40 is better than gate_30: {(treat_7_samples > control_7_samples).mean():.4%}")
print(f"At 95% confident level for relative effect: [{np.percentile(relative_effect_7, 2.5):.2%}, {np.percentile(relative_effect_7, 97.5):.2%}]")

DAY 1 Retention _____________________
Control (gate_30) posterior mean: 44.8204%
Treatment (gate_40) posterior mean: 44.2285%
Absolute difference: -0.5920% percentage points
Relative change: -1.32%
Probability that gate_40 is BETTER than gate_30: 3.7280%
95% Credible Interval for relative effect: [-2.75%, 0.14%]


DAY 7 Retention _____________________
Control (gate_30) posterior mean: 19.0213%
Treatment (gate_40) posterior mean: 18.2032%
Absolute difference: -0.8181% percentage points
Relative change: -4.29%
Probability that gate_40 is BETTER than gate_30: 0.0830%
95% Credible Interval for relative effect: [-6.90%, -1.64%]
